# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [2]:
import pandas as pd
import numpy as np

import tensorflow as tf # Library to construct neural newtorks. Low level library
from tensorflow import keras # High level library to handle tensorflow.
from tensorflow.keras.layers import Flatten, Dense, Dropout # Types of layers to use in our neural network.

import warnings
warnings.filterwarnings("ignore")


from google.colab import drive
drive.mount("/content/drive/")


Mounted at /content/drive/


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:

data = pd.read_csv("/content/drive/MyDrive/IRONHACK/lab-deep-learning/your-code/tic-tac-toe.csv")

In [33]:
for col in data.columns:
  data_x[col] = np.where(data[col].isin(['x']),1,0)
data_x['class'] = data['class']
data_x
data_x['class'] = np.where(data[col].isin([True]),1,0)
display(data.head())
display(data_x.head())

# only 'x' matters for the classification. However it is not clear - should we drop other values (as below), or we should 
# convert them to numerical as well (for example as -1 for 'o' and 0 for 'p') and feed it to neural network (i tried but 
# the accuracy is very low). We can also dummify the columns (and get 27 columns with input data).

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,1,1,1,1,0,0,1,0,0,1
1,1,1,1,1,0,0,0,1,0,1
2,1,1,1,1,0,0,0,0,1,1
3,1,1,1,1,0,0,0,0,0,1
4,1,1,1,1,0,0,0,0,0,1


In [17]:
X = data_x.iloc[:,0:9]
y = data_x[['class']]

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=7)

print(X_train.shape)
print(X_test.shape)

(766, 9)
(192, 9)


In [59]:
from tensorflow.keras.callbacks import EarlyStopping 
from tensorflow.keras.callbacks import ModelCheckpoint

early_stopping = EarlyStopping(patience=5) 

model_c1 = keras.Sequential()

model_c1.add(Dense(units = 8,input_dim = X_train.shape[1],activation='relu')) #8 possible ways to draw of a line of 'x'

model_c1.add(Dense(units = 1, activation='relu'))

model_c1.add(Dense(units = 1, activation='sigmoid'))


model_c1.compile(optimizer='adam', 
              loss='binary_crossentropy', # i tried sparse_categorical_crossentropy but it requires labels to be in range [0,1)
              metrics=['accuracy']
              )


model_c1.summary()

checkpoint_path = '/content/drive/MyDrive/IRONHACK/lab-deep-learning/your-code/cls-nn1.hdf5'
checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    save_freq='epoch',
    save_weights_only=False,
    verbose=1
)

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 8)                 80        
                                                                 
 dense_45 (Dense)            (None, 1)                 9         
                                                                 
 dense_46 (Dense)            (None, 1)                 2         
                                                                 
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________


In [60]:
history_c1 = model_c1.fit(
    X_train, 
    y_train, 
    epochs=150, 
    validation_split=0.20, 
    batch_size=10, 
    verbose=2,
    callbacks=[early_stopping,checkpoint]
)

Epoch 1/150

Epoch 1: saving model to /content/drive/MyDrive/IRONHACK/lab-deep-learning/your-code/cls-nn1.hdf5
62/62 - 1s - loss: 0.6317 - accuracy: 0.6601 - val_loss: 0.5878 - val_accuracy: 0.6558 - 841ms/epoch - 14ms/step
Epoch 2/150

Epoch 2: saving model to /content/drive/MyDrive/IRONHACK/lab-deep-learning/your-code/cls-nn1.hdf5
62/62 - 0s - loss: 0.6152 - accuracy: 0.6634 - val_loss: 0.5758 - val_accuracy: 0.6558 - 139ms/epoch - 2ms/step
Epoch 3/150

Epoch 3: saving model to /content/drive/MyDrive/IRONHACK/lab-deep-learning/your-code/cls-nn1.hdf5
62/62 - 0s - loss: 0.6049 - accuracy: 0.6634 - val_loss: 0.5659 - val_accuracy: 0.6558 - 146ms/epoch - 2ms/step
Epoch 4/150

Epoch 4: saving model to /content/drive/MyDrive/IRONHACK/lab-deep-learning/your-code/cls-nn1.hdf5
62/62 - 0s - loss: 0.5983 - accuracy: 0.6634 - val_loss: 0.5585 - val_accuracy: 0.6558 - 143ms/epoch - 2ms/step
Epoch 5/150

Epoch 5: saving model to /content/drive/MyDrive/IRONHACK/lab-deep-learning/your-code/cls-nn1.h

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [41]:
# your code here

def model_performance_class(y_train, y_pred_train, y_test, y_pred_test):

    from sklearn.metrics import cohen_kappa_score, classification_report 


    print("Results obtained for the TRAIN SET")
    print("==================================")
    print("The Cohen's Kappa is: {:.2f}".format(cohen_kappa_score(y_train, y_pred_train)))
    print(classification_report(y_train, y_pred_train))
    print("==================================")
    print("Results obtained for the TEST SET")
    print("The Cohen's Kappa is: {:.2f}".format(cohen_kappa_score(y_test, y_pred_test)))
    print(classification_report(y_test, y_pred_test))

y_pred_train = model_c1.predict(X_train).round(0) ## rounded to 1
y_pred_test  = model_c1.predict(X_test).round(0) ## rounded to 1

y_train = np.array(y_train).reshape(-1,1)
y_test  = np.array(y_test).reshape(-1,1)

results = model_performance_class(y_train, y_pred_train, y_test, y_pred_test)
results

Results obtained for the TRAIN SET
The Cohen's Kappa is: 0.87
              precision    recall  f1-score   support

           0       1.00      0.83      0.91       259
           1       0.92      1.00      0.96       507

    accuracy                           0.94       766
   macro avg       0.96      0.92      0.93       766
weighted avg       0.95      0.94      0.94       766

Results obtained for the TEST SET
The Cohen's Kappa is: 0.82
              precision    recall  f1-score   support

           0       1.00      0.78      0.88        73
           1       0.88      1.00      0.94       119

    accuracy                           0.92       192
   macro avg       0.94      0.89      0.91       192
weighted avg       0.93      0.92      0.91       192



## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [54]:
# your code here

opt = tf.keras.optimizers.Adam(learning_rate=0.03)

model_c2 = keras.Sequential()

model_c2.add(Dense(units = 8,input_dim = X_train.shape[1],activation='relu')) 

model_c2.add(Dense(units = 1, activation='relu'))

model_c2.add(Dense(units = 1, activation='sigmoid'))


model_c2.compile(optimizer=opt, 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )


#model_c2.summary()

checkpoint_path = '/content/drive/MyDrive/IRONHACK/lab-deep-learning/your-code/cls-nn2.hdf5'
checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    save_freq='epoch',
    save_weights_only=False,
    verbose=1
)



In [55]:
history_c2 = model_c2.fit(
    X_train, 
    y_train, 
    epochs=150, 
    validation_split=0.20, 
    batch_size=10, 
    verbose=1,
    callbacks=[early_stopping,checkpoint]
)

Epoch 1/150
27/62 [============>.................] - ETA: 0s - loss: 0.6586 - accuracy: 0.6444 
Epoch 1: saving model to /content/drive/MyDrive/IRONHACK/lab-deep-learning/your-code/cls-nn2.hdf5
62/62 [==============================] - 1s 7ms/step - loss: 0.6240 - accuracy: 0.6895 - val_loss: 0.5449 - val_accuracy: 0.7597
Epoch 2/150
38/62 [=================>............] - ETA: 0s - loss: 0.5271 - accuracy: 0.7632
Epoch 2: saving model to /content/drive/MyDrive/IRONHACK/lab-deep-learning/your-code/cls-nn2.hdf5
62/62 [==============================] - 0s 2ms/step - loss: 0.5299 - accuracy: 0.7631 - val_loss: 0.4601 - val_accuracy: 0.8117
Epoch 3/150
36/62 [================>.............] - ETA: 0s - loss: 0.5032 - accuracy: 0.7833
Epoch 3: saving model to /content/drive/MyDrive/IRONHACK/lab-deep-learning/your-code/cls-nn2.hdf5
62/62 [==============================] - 0s 3ms/step - loss: 0.5154 - accuracy: 0.7745 - val_loss: 0.4904 - val_accuracy: 0.7922
Epoch 4/150
37/62 [==============

In [56]:
# your code here

def model_performance_class(y_train, y_pred_train, y_test, y_pred_test):

    from sklearn.metrics import cohen_kappa_score, classification_report 


    print("Results obtained for the TRAIN SET")
    print("==================================")
    print("The Cohen's Kappa is: {:.2f}".format(cohen_kappa_score(y_train, y_pred_train)))
    print(classification_report(y_train, y_pred_train))
    print("==================================")
    print("Results obtained for the TEST SET")
    print("The Cohen's Kappa is: {:.2f}".format(cohen_kappa_score(y_test, y_pred_test)))
    print(classification_report(y_test, y_pred_test))

y_pred_train = model_c2.predict(X_train).round(0) ## rounded
y_pred_test  = model_c2.predict(X_test).round(0) ## rounded 

y_train = np.array(y_train).reshape(-1,1)
y_test  = np.array(y_test).reshape(-1,1)

results = model_performance_class(y_train, y_pred_train, y_test, y_pred_test)
results

Results obtained for the TRAIN SET
The Cohen's Kappa is: 0.68
              precision    recall  f1-score   support

           0       1.00      0.61      0.76       259
           1       0.84      1.00      0.91       507

    accuracy                           0.87       766
   macro avg       0.92      0.81      0.84       766
weighted avg       0.89      0.87      0.86       766

Results obtained for the TEST SET
The Cohen's Kappa is: 0.55
              precision    recall  f1-score   support

           0       1.00      0.49      0.66        73
           1       0.76      1.00      0.87       119

    accuracy                           0.81       192
   macro avg       0.88      0.75      0.76       192
weighted avg       0.85      0.81      0.79       192



**Which approach(es) did you find helpful to improve your model performance?**

In [56]:
## increasing learning rate usually lead to decrease in accuracy
## increasing the number of epochs is generally positive for accuracy